# PySAL with Numba Performance Test

In [1]:
import sys
import libpysal as lps
import numpy as np
import geopandas as gpd
import esda
import time

## Testing method

In [2]:


"""
Usage: 
python3 perf_pysal.py FILE_PATH VARIABLE_NAME PERMUTATIONS CPU_THREADS
python3 perf_pysal.py ./data/natregimes.shp HR60 999 1

data_path = sys.argv[1]
var_name = sys.argv[2]
perms = int(sys.argv[3])
cpu_threads = int(sys.argv[4])
"""

def test(data_path, var_name, perms, cpu_threads):
    np.random.seed(12345)


    # prepare the data and weights
    gdf = gpd.read_file(data_path)
    x = gdf[var_name]


    if data_path == './data/Chicago_parcels_points.shp':
        w = lps.weights.KNN.from_dataframe(gdf, k=10)
    else:
        w = lps.weights.Queen.from_dataframe(gdf)
        # remove islands
        if len(w.islands) > 0:
            df_noi = gdf[~gdf.index.isin(w.islands)]
            w = lps.weights.w_subset(w, df_noi.index.tolist())
            x = df_noi[var_name]
    w.transform = 'r'


    # function to execute and time
    start_time = time.time()
    li = esda.moran.Moran_Local(x, w, permutations=perms, n_jobs=cpu_threads, keep_simulations=False)
    run_time = time.time() - start_time

    print("{0} {1} {2}: {3} seconds".format(data_path, perms, cpu_threads, run_time))
    return run_time

## Data

In [3]:
test_ds = {'./data/natregimes.shp' : 'HR60',
          './data/us-sdoh-2014.shp' : 'ep_unem',
          './data/NYC_Area2010_2data.shp' : 'CE01_02',
          #'./data/Chicago_parcels_points.shp' : 'EstBuild'
          }

perm_set = [999, 9999, 99999]

threads_set = [1, 8, 16]

## Test

In [4]:
try:
    import numba
    print(f"Numba version installed: {numba.__version__}")
except:
    print("Numba not installed")

Numba not installed


In [5]:
%%time
t = test('./data/us-sdoh-2014.shp', 'ep_unem', 999, 1)
print(f"\t{t} seconds.")

/Users/xun/Github/libgeoda_paper/venv/lib/python3.6/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 30 disconnected components.
 There are 15 islands with ids: 6540, 6583, 7933, 19288, 23844, 33314, 35600, 38699, 39317, 53310, 58715, 58849, 69632, 70049, 70050.
  warnings.warn(message)
/Users/xun/Github/libgeoda_paper/venv/lib/python3.6/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 15 disconnected components.
  warnings.warn(message)


./data/us-sdoh-2014.shp 999 1: 25.524923086166382 seconds
	25.524923086166382 seconds.
CPU times: user 4min 26s, sys: 50.2 s, total: 5min 16s
Wall time: 1min 13s
